In [25]:
pip install tqdm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.3 MB 3.0 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.3 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available: True
Current device: 0
Device name: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [1]:
pip install transformers datasets peft accelerate bitsandbytes


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install fsspec==2023.9.2 datasets


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

df = pd.read_csv("C:/Users/alpha/Downloads/mit.csv", encoding="ISO-8859-1")  # or encoding="utf-8" if needed
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)


In [4]:
def format_example(example):
    return {
        "text": f"### User: {example['question']}\n### Assistant: {example['answer']}"
    }

dataset = dataset.map(format_example)


Map:   0%|          | 0/588 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [9]:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 588
    })
    test: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 66
    })
})


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Or mistralai/Mistral-7B-Instruct-v0.1
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [6]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/588 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType

# Set LoRA config
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)

# Apply LoRA to your base model
model = get_peft_model(model, peft_config)




In [24]:
pip install trl


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.4 MB 4.8 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/10.4 MB 5.0 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.4 MB 5.1 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.4 MB 6.0 MB/s eta 0:00:01
   ------------------------ --------------- 6.3/10.4 MB 6.3 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.4 MB 6.8 MB/s eta 0:00:01
   -------------------------------------- - 10.0/10.4 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.3 MB 10.3 MB/s eta 0:00:03
   ------- -------------------------------- 5.0/25.3 MB 12.1 MB/s eta 0:00:02
   ----------

  You can safely remove it manually.


In [1]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="LEVI",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    dataset_text_field="text",  # This tells it to use the 'text' column
)


NameError: name 'model' is not defined

In [ ]:
trainer.train()
trainer.save_model("levi.1")
tokenizer.save_pretrained("levi.1")
